Notebook para a divisão k-fold

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
def read_file_as_df(file_name):
    import pandas as pd
    import csv

    import sys
    import pandas as pd

    maxInt = sys.maxsize

    while True:
        # decrease the maxInt value by factor 10 
        # as long as the OverflowError occurs.

        try:
            csv.field_size_limit(maxInt)
            break
        except OverflowError:
            maxInt = int(maxInt/10)

    file = []
    col = []

    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        line_count = 0
        for row in csv_reader:
            if line_count==0:
                for r in row:
                    col.append(r)
                line_count+=1
            else:
                line = []
                for r in row:
                    line.append(r)
                file.append(line)
                line_count += 1

            
    df = pd.DataFrame(file, columns = col)
    return df


In [4]:
df = read_file_as_df('C:/Users/alici/Documents/tcc/code/partições/b5/b5_post_concat.csv')

In [5]:
df.head()

,User_ID,inventory,exp,ActualGender,age,TI,religiousity,course,extraversion,agreeableness,...,activity,altruism,compliance,order,selfdiscipline,ansiety,depression,aesthetics,ideas,Text
0,17,facebook,0,m,24.0,yes,2.0,1,"2,75","4,56",...,"3,5","4,75",4,"4,5","3,2","2,75",2,2,4,Ajude uma pesquisa de mestrado e descubra mais...
1,19,facebook,2,f,39.0,yes,3.0,0,"4,13","3,22",...,"3,5",4,"2,33",2,"3,6","3,75","3,5",4,3,"saudades sempre linda # "" E aí beleza queremo..."
2,20,facebook,2,f,31.0,no,4.0,0,"4,25","3,67",...,"3,5","4,5","2,67",4,5,3,"1,5","3,67","3,2",Sem mais . . . # verdade # $LAUGH$ pior que ...
3,21,facebook,2,m,28.0,yes,3.0,0,"3,38","3,56",...,"3,5","3,5","3,33",2,3,"4,5",3,"2,33","4,8",Excelente notícia . : # E o silêncio reina em...
4,22,facebook,2,f,31.0,no,4.0,0,"4,13","4,11",...,"3,5","4,75",3,"3,5","3,2","3,75","1,5","4,33","3,8",""" Verdade ! Mas , ainda faltam alguns nomes ne..."


In [6]:
df.shape

(1082, 24)

In [7]:
df.dropna(axis=1, inplace=True)
df.drop_duplicates(inplace=True)

In [8]:
df.shape

(1082, 24)

In [9]:
df.columns

Index(['User_ID', 'inventory', 'exp', 'ActualGender', 'age', 'TI',
       'religiousity', 'course', 'extraversion', 'agreeableness',
       'conscientiousness', 'neuroticism', 'openness', 'assertiveness',
       'activity', 'altruism', 'compliance', 'order', 'selfdiscipline',
       'ansiety', 'depression', 'aesthetics', 'ideas', 'Text'],
      dtype='object')

In [10]:
df = df.drop(columns=['TI',
       'religiousity', 'course', 'extraversion', 'agreeableness',
       'conscientiousness', 'neuroticism', 'openness', 'assertiveness',
       'activity', 'altruism', 'compliance', 'order', 'selfdiscipline',
       'ansiety', 'depression', 'aesthetics', 'ideas', 'inventory', 'exp'])

In [11]:
df.columns

Index(['User_ID', 'ActualGender', 'age', 'Text'], dtype='object')

In [12]:
df.shape

(1082, 4)

In [13]:
df['ActualGender'] = df['ActualGender'].replace('', np.nan)
df['age'] = df['age'].replace('', np.nan)
df['Text'] = df['Text'].replace('', np.nan)

In [14]:
df[df['Text'].isna()].shape

(62, 4)

In [15]:
df = df.dropna(subset=['age', 'Text'])

In [16]:
df.shape

(754, 4)

In [17]:
def ageGroups(text):
    age = int(float(text))
    if age >=18 and age<=20:
        return 1
    elif age >=23 and age<=25:
        return 2
    elif age >= 28 and age<=61:
        return 3
    else:
        return 4

In [20]:
df["AgeClass"] = df["age"].apply(ageGroups)

In [24]:
df.drop(df[df['AgeClass'] == 4].index, inplace = True)

In [25]:
df.shape

(517, 5)

In [26]:
def genderGroups(text):
    if(text == "f"):
        return 1
    else:
        return 0

In [27]:
df["GenderClass"] = df["ActualGender"].apply(genderGroups)

In [28]:
df.dropna(axis=1, inplace=True)
df.dropna(axis=1, inplace=True)

df.drop_duplicates(inplace=True)
df.drop_duplicates(inplace=True)

In [29]:
df.shape

(517, 6)

In [34]:
def GroupAgeAndGender(df):
    ageGender = []
    for _, line in df.iterrows():
        if line["AgeClass"]==1 and line["GenderClass"]==1:
            ageGender.append(1)
        elif line["AgeClass"]==1 and line["GenderClass"]==0:
            ageGender.append(2)
        elif line["AgeClass"]==2 and line["GenderClass"]==1:
            ageGender.append(3)
        elif line["AgeClass"]==2 and line["GenderClass"]==0:
            ageGender.append(4)
        elif line["AgeClass"]==3 and line["GenderClass"]==1:
            ageGender.append(5)
        elif line["AgeClass"]==3 and line["GenderClass"]==0:
            ageGender.append(6)
    df["AgeGender"] = ageGender

In [38]:
df.reset_index(drop=True, inplace=True)

In [35]:
GroupAgeAndGender(df)

In [36]:
from sklearn.model_selection import StratifiedKFold

In [37]:
kf_strat = StratifiedKFold(n_splits=10, shuffle=True)

In [39]:
x = df
y = df["AgeGender"]

In [40]:
def getsplits_strat(df, samples):
     kfold_list = []
     for train_index, test_index in kf_strat.split(df, samples):
          print(len(train_index), len(test_index))
          dftrain = df.iloc[train_index]
          dftest = df.iloc[test_index]
          kfold_list.append([dftrain, dftest])
     return kfold_list

In [41]:
kfold_list = getsplits_strat(x, y)

465 52
465 52
465 52
465 52
465 52
465 52
465 52
466 51
466 51
466 51


In [42]:
for train, test in kfold_list:
    train_fem = train[train["GenderClass"]==1]
    train_masc = train[train["GenderClass"]==0]
    print(train_fem["AgeClass"].value_counts())
    print(train_masc["AgeClass"].value_counts())

    test_fem = test[test["GenderClass"]==1]
    test_masc = test[test["GenderClass"]==0]
    print(test_fem["AgeClass"].value_counts())
    print(test_masc["AgeClass"].value_counts())

1    105
2     85
3     70
Name: AgeClass, dtype: int64
2    85
3    61
1    59
Name: AgeClass, dtype: int64
1    11
2     9
3     8
Name: AgeClass, dtype: int64
2    10
3     7
1     7
Name: AgeClass, dtype: int64
1    104
2     84
3     70
Name: AgeClass, dtype: int64
2    85
3    62
1    60
Name: AgeClass, dtype: int64
1    12
2    10
3     8
Name: AgeClass, dtype: int64
2    10
3     6
1     6
Name: AgeClass, dtype: int64
1    104
2     84
3     70
Name: AgeClass, dtype: int64
2    85
3    62
1    60
Name: AgeClass, dtype: int64
1    12
2    10
3     8
Name: AgeClass, dtype: int64
2    10
1     6
3     6
Name: AgeClass, dtype: int64
1    104
2     84
3     71
Name: AgeClass, dtype: int64
2    85
3    61
1    60
Name: AgeClass, dtype: int64
1    12
2    10
3     7
Name: AgeClass, dtype: int64
2    10
3     7
1     6
Name: AgeClass, dtype: int64
1    104
2     84
3     71
Name: AgeClass, dtype: int64
2    85
3    61
1    60
Name: AgeClass, dtype: int64
1    12
2    10
3     7
Name: A

### Salvar em arquivo

In [43]:
def create_split_csv(l, path):
    import os
    import shutil

    root_folder = path
    folder_name = "k_"
    file_name1 = "train"
    file_name2 = "test"
    k = 1

    for training, test in l:
        path = os.path.join(root_folder, (folder_name+str(k)))
        if(os.path.exists(path)):
            shutil.rmtree(path)
        os.mkdir(path)

        training.to_csv((path+"/"+file_name1+".csv"), index = False, header=True)
        test.to_csv((path+"/"+file_name2+".csv"), index = False, header=True)

        k+=1

In [44]:
create_split_csv(kfold_list, "C:/Users/alici/Documents/tcc/github2/tcc-v2/b5/particoes")